In [ ]:
# Inside your Databricks Notebook
db_pass = dbutils.secrets.get(scope="sales_project", key="pg_pass")

In [ ]:
# Databricks Notebook: transform_sales
from pyspark.sql.functions import col, sum

# 1. Load data (Airflow will upload the file to this location)
file_path = "/FileStore/tables/sales_data.csv"
df = spark.read.format("csv").option("header", "true").load(file_path)

# 2. Transform: Calculate total revenue by product
transformed_df = df.withColumn("amount", col("amount").cast("double")) \
                   .groupBy("product_name") \
                   .agg(sum("amount").alias("total_revenue"))

# 3. Save as a Delta Table
transformed_df.write.format("delta").mode("overwrite").saveAsTable("gold_sales_summary")